In [1]:
import pandas as pd
import numpy as np
import sys
import os
import bs4
from IPython.display import display_html,clear_output, HTML
import re
from datetime import datetime
import ast
import itertools
from tqdm import tqdm,trange
from file_tools import *
from request_tools import *
from parse_tools import *
from load_tools import *
from model_tools import *

pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 30)
pd.set_option('compute.use_numexpr', False)

import warnings
from pandas.errors import PerformanceWarning
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning
pd.options.mode.use_inf_as_na = True
warnings.filterwarnings('ignore', category=PerformanceWarning)
warnings.filterwarnings('ignore', category=DataConversionWarning)
warnings.filterwarnings('ignore', category=ConvergenceWarning)

import pandas as pd
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif, RFE, mutual_info_regression
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from scipy.stats import pearsonr
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import SVR


In [2]:
df_index = pd.read_csv('./05-data-indexed/team_opp_games_full_boxscores_index.csv')
df_index = df_index.set_index(['Boxscores_id','Team_id'],drop=False)
df_index

Boxscores_id   
Boxscores_id                 Team_id                                              
/boxscores/196710130OAK.html /teams/ANA/1968.html  /boxscores/196710130OAK.html  \
/boxscores/196710160DLC.html /teams/DLC/1968.html  /boxscores/196710160DLC.html   
/boxscores/196710150DNR.html /teams/DNR/1968.html  /boxscores/196710150DNR.html   
/boxscores/196710170DNR.html /teams/HSM/1968.html  /boxscores/196710170DNR.html   
/boxscores/196710140INA.html /teams/INA/1968.html  /boxscores/196710140INA.html   
                             /teams/KEN/1968.html  /boxscores/196710140INA.html   
/boxscores/196710220MNM.html /teams/MNM/1968.html  /boxscores/196710220MNM.html   
/boxscores/196710230NJA.html /teams/NJA/1968.html  /boxscores/196710230NJA.html   
/boxscores/196710190NOB.html /teams/NOB/1968.html  /boxscores/196710190NOB.html   
/boxscores/196710130OAK.html /teams/OAK/1968.html  /boxscores/196710130OAK.html   
/boxscores/196710230NJA.html /teams/PTP/1968.html  /boxscores/196710230NJA.html   
/boxscores/196710150DNR.html /teams/ANA/1968.html  /boxscores/196710150DNR.html   
/boxscores/196710220DLC.html /teams/DLC/1968.html  /boxscores/196710220DLC.html   
/boxscores/196710170DNR.html /teams/DNR/1968.html  /boxscores/196710170DNR.html   
/boxscores/196710190NOB.html /teams/HSM/1968.html  /boxscores/196710190NOB.html   
...                                                                         ...   
/boxscores/202311250BRK.html /teams/MIA/2024.html  /boxscores/202311250BRK.html   
/boxscores/202311260MIL.html /teams/MIL/2024.html  /boxscores/202311260MIL.html   
/boxscores/202311250UTA.html /teams/NOP/2024.html  /boxscores/202311250UTA.html   
/boxscores/202311260ORL.html /teams/ORL/2024.html  /boxscores/202311260ORL.html   
/boxscores/202311270PHI.html /teams/PHI/2024.html  /boxscores/202311270PHI.html   
/boxscores/202311260NYK.html /teams/PHO/2024.html  /boxscores/202311260NYK.html   
/boxscores/202311270IND.html /teams/POR/2024.html  /boxscores/202311270IND.html   
/boxscores/202311260DEN.html /teams/SAS/2024.html  /boxscores/202311260DEN.html   
/boxscores/202311260CLE.html /teams/TOR/2024.html  /boxscores/202311260CLE.html   
/boxscores/202311270UTA.html /teams/UTA/2024.html  /boxscores/202311270UTA.html   
/boxscores/202311270DET.html /teams/WAS/2024.html  /boxscores/202311270DET.html   
/boxscores/202311260BRK.html /teams/CHI/2024.html  /boxscores/202311260BRK.html   
/boxscores/202311270LAC.html /teams/DEN/2024.html  /boxscores/202311270LAC.html   
/boxscores/202311270PHI.html /teams/LAL/2024.html  /boxscores/202311270PHI.html   
/boxscores/202311270UTA.html /teams/NOP/2024.html  /boxscores/202311270UTA.html   

                                                                Team_id   
Boxscores_id                 Team_id                                      
/boxscores/196710130OAK.html /teams/ANA/1968.html  /teams/ANA/1968.html  \
/boxscores/196710160DLC.html /teams/DLC/1968.html  /teams/DLC/1968.html   
/boxscores/196710150DNR.html /teams/DNR/1968.html  /teams/DNR/1968.html   
/boxscores/196710170DNR.html /teams/HSM/1968.html  /teams/HSM/1968.html   
/boxscores/196710140INA.html /teams/INA/1968.html  /teams/INA/1968.html   
                             /teams/KEN/1968.html  /teams/KEN/1968.html   
/boxscores/196710220MNM.html /teams/MNM/1968.html  /teams/MNM/1968.html   
/boxscores/196710230NJA.html /teams/NJA/1968.html  /teams/NJA/1968.html   
/boxscores/196710190NOB.html /teams/NOB/1968.html  /teams/NOB/1968.html   
/boxscores/196710130OAK.html /teams/OAK/1968.html  /teams/OAK/1968.html   
/boxscores/196710230NJA.html /teams/PTP/1968.html  /teams/PTP/1968.html   
/boxscores/196710150DNR.html /teams/ANA/1968.html  /teams/ANA/1968.html   
/boxscores/196710220DLC.html /teams/DLC/1968.html  /teams/DLC/1968.html   
/boxscores/196710170DNR.html /teams/DNR/1968.html  /teams/DNR/1968.html   
/boxscores/196710190NOB.html /teams/HSM/1968.html  /teams/HSM/1968.html   
...                                                 

In [3]:
BXSC_PLYR_H1 =  pd.read_csv('./03-data-aggregated/boxscores/boxscores_players_H1.csv', index_col=[0,1,2])
BXSC_TEAM_H1 = pd.read_csv('./03-data-aggregated/boxscores/boxscores_teams_H1.csv', index_col=[0,1])


BXSC_PLYR_H1_SEASON = BXSC_PLYR_H1.index.to_frame()['team_id'].str[-9:-5].astype(int)
BXSC_TEAM_H1_SEASON = BXSC_TEAM_H1.index.to_frame()['team_id'].str[-9:-5].astype(int)
ELIGIBLE_SEASONS = BXSC_PLYR_H1_SEASON[BXSC_PLYR_H1_SEASON <= 2023].index
ELIGIBLE_TM_SEASONS = BXSC_TEAM_H1_SEASON[BXSC_TEAM_H1_SEASON <= 2023].index.intersection(df_index.index)

BXSC_PLYR_H1 = BXSC_PLYR_H1.loc[ELIGIBLE_SEASONS]
BXSC_TEAM_H1 = BXSC_TEAM_H1.loc[ELIGIBLE_TM_SEASONS]

BXSC_PLYR_H2 =  pd.read_csv('./03-data-aggregated/boxscores/boxscores_players_H2.csv', index_col=[0,1,2]).loc[ELIGIBLE_SEASONS]
BXSC_PLYR_Q1 =  pd.read_csv('./03-data-aggregated/boxscores/boxscores_players_Q1.csv', index_col=[0,1,2]).loc[ELIGIBLE_SEASONS]
BXSC_PLYR_Q2 =  pd.read_csv('./03-data-aggregated/boxscores/boxscores_players_Q2.csv', index_col=[0,1,2]).loc[ELIGIBLE_SEASONS]
BXSC_PLYR_Q3 =  pd.read_csv('./03-data-aggregated/boxscores/boxscores_players_Q3.csv', index_col=[0,1,2]).loc[ELIGIBLE_SEASONS]
BXSC_PLYR_Q4 =  pd.read_csv('./03-data-aggregated/boxscores/boxscores_players_Q4.csv', index_col=[0,1,2]).loc[ELIGIBLE_SEASONS]
BXSC_PLYR_GM = pd.read_csv('./03-data-aggregated/boxscores/boxscores_players_G.csv', index_col=[0,1,2]).loc[ELIGIBLE_SEASONS] # Target
BXSC_TEAM_Q3 = pd.read_csv('./03-data-aggregated/boxscores/boxscores_teams_Q3.csv', index_col=[0,1]).loc[ELIGIBLE_TM_SEASONS]
BXSC_TEAM_GM = pd.read_csv('./03-data-aggregated/boxscores/boxscores_teams_G.csv', index_col=[0,1]).loc[ELIGIBLE_TM_SEASONS]

# BXSC_TEAM_Q4 = pd.read_csv('./03-data-aggregated/boxscores/boxscores_teams_Q4.csv', index_col=[0,1]).loc[ELIGIBLE_TM_SEASONS]
# BXSC_TEAM_H2 = pd.read_csv('./03-data-aggregated/boxscores/boxscores_teams_H2.csv', index_col=[0,1]).loc[ELIGIBLE_TM_SEASONS]

BXSC_PLYR_3Q = BXSC_PLYR_H1 + BXSC_PLYR_Q3
BXSC_PLYR_3Q['fg%'] = (BXSC_PLYR_3Q['fg']/BXSC_PLYR_3Q['fga']).fillna(0)
BXSC_PLYR_3Q['3p%'] = (BXSC_PLYR_3Q['3p']/BXSC_PLYR_3Q['3pa']).fillna(0)
BXSC_PLYR_3Q['ft%'] = (BXSC_PLYR_3Q['ft']/BXSC_PLYR_3Q['fta']).fillna(0)

BXSC_TEAM_3Q = BXSC_TEAM_H1 + BXSC_TEAM_Q3
BXSC_TEAM_3Q['fg%'] = (BXSC_TEAM_3Q['fg']/BXSC_TEAM_3Q['fga']).fillna(0)
BXSC_TEAM_3Q['3p%'] = (BXSC_TEAM_3Q['3p']/BXSC_TEAM_3Q['3pa']).fillna(0)
BXSC_TEAM_3Q['ft%'] = (BXSC_TEAM_3Q['ft']/BXSC_TEAM_3Q['fta']).fillna(0)

# BXSC_PLYR_3Q
# BXSC_TEAM_3Q
# # Filter out rows
# BXSC_PLYR_GM = BXSC_PLYR_GM.loc[BXSC_PLYR_H1.index] # Since quarter-by-quarter data is not available until 1996/1997, we only select the years for which we have both types of data
# # Next, we filter only players who played at least 5 minutes in the first half and at least 20 minutes in the game
# COND_MIN_MP = (BXSC_PLYR_H1['mp'] > 5) & (BXSC_PLYR_GM['mp'] >= 20)
# BXSC_PLYR_H1 = BXSC_PLYR_H1.loc[COND_MIN_MP]
# BXSC_PLYR_GM = BXSC_PLYR_GM.loc[COND_MIN_MP]

In [4]:
BXSC_PLYR_H1_Q12 = pd.concat([BXSC_PLYR_H1,BXSC_PLYR_Q1, BXSC_PLYR_Q2, BXSC_PLYR_Q3], axis=1,keys=['H1','Q1','Q2'])
BXSC_PLYR_H1_Q12

OPP_IDX = pd.MultiIndex.from_frame(df_index.loc[BXSC_TEAM_H1.index][['Boxscores_id','Opp_id']],names=['boxscore_id','team_id'])

# H1 stats
BXSC_OPP_H1 = pd.DataFrame(data=BXSC_TEAM_H1.copy().loc[OPP_IDX].values,index=BXSC_TEAM_H1.index,columns=BXSC_TEAM_H1.columns)
BXSC_DIFF_H1 = BXSC_TEAM_H1 - BXSC_OPP_H1
BXSC_TM_OPP_DIFF_H1 = pd.concat([BXSC_TEAM_H1,BXSC_OPP_H1,BXSC_DIFF_H1],axis=1,keys=['Team_H1','Opp_H1','Diff_H1'])
BXSC_TM_OPP_DIFF_H1

# Reset the indices
# Merge the DataFrames
BXSC_PLYR_TM_1ST_HALF = pd.merge(BXSC_PLYR_H1_Q12.reset_index(), BXSC_TM_OPP_DIFF_H1.reset_index(), on=['boxscore_id', 'team_id'],how='inner').set_index(['boxscore_id', 'team_id','player_id'])
# Set the index back to the desired multi-index
BXSC_PLYR_TM_1ST_HALF


H1   
                                                                                    mp   
boxscore_id                  team_id              player_id                              
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/smithja05.html  15.033333  \
                                                  /players/c/chrisjo01.html  12.183333   
                                                  /players/g/greenja05.html  15.150000   
                                                  /players/s/sengual01.html  13.366667   
                                                  /players/m/martike04.html  10.866667   
                                                  /players/e/easonta01.html  13.133333   
                                                  /players/w/washity02.html   8.850000   
                                                  /players/n/nixda01.html    11.816667   
                                                  /players/k/kaminfr01.html   8.966667   
                                                  /players/m/marjabo01.html   4.733333   
                                                  /players/g/garubus01.html   5.900000   
                                                  /players/a/augusdj01.html   0.000000   
                                                  /players/c/caulewi01.html   0.000000   
                                                  /players/d/daysda01.html    0.000000   
                                                  /players/h/hudgitr01.html   0.000000   
...                                                                                ...   
/boxscores/199611010BOS.html /teams/CHI/1997.html /players/k/kerrst01.html    8.500000   
                                                  /players/w/wennibi01.html   5.616667   
                                                  /players/p/parisro01.html   0.000000   
                                                  /players/b/brownra02.html   2.650000   
                                                  /players/s/simpkdi01.html   4.700000   
                                                  /players/b/buechju01.html   0.600000   
                             /teams/BOS/1997.html /players/w/wesleda01.html  18.483333   
                                                  /players/f/foxri01.html    16.566667   
                                                  /players/e/ellispe01.html  15.050000   
                                                  /players/w/walkean02.html  18.250000   
                                                  /players/w/willier01.html  12.700000   
                                                  /players/b/barroda01.html  16.316667   
                                                  /players/d/dayto01.html     9.466667   
                                                  /players/b/brickfr01.html  11.300000   
                                                  /players/s/szabobr01.html   1.866667   

                                                                                   
                                                                              fg   
boxscore_id                  team_id              player_id                        
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/smithja05.html  4.0  \
                                                  /players/c/chrisjo01.html  3.0   
                                                  /players/g/greenja05.html  2.0   
                                                  /players/s/sengual01.html  0.0   
                                                  /players/m/martike04.html  1.0   
                                                  /players/e/easonta01.html  3.0   
                                                  /players/w/washity02.html  3.0   
                                                  /players/n/nixda01.html    1.0   
                                                  /players/k/kaminfr01.html  1.0   
               

2nd Half Stats

In [5]:
OPP_IDX = pd.MultiIndex.from_frame(df_index.loc[BXSC_TEAM_H1.index][['Boxscores_id','Opp_id']],names=['boxscore_id','team_id'])
# 3Q stats
BXSC_OPP_3Q = pd.DataFrame(data=BXSC_TEAM_3Q.copy().loc[OPP_IDX].values,index=BXSC_TEAM_3Q.index,columns=BXSC_TEAM_3Q.columns)
BXSC_DIFF_3Q = BXSC_TEAM_3Q - BXSC_OPP_3Q
BXSC_TM_OPP_DIFF_3Q = pd.concat([BXSC_TEAM_3Q,BXSC_OPP_3Q,BXSC_DIFF_3Q],axis=1,keys=['Team_3Q','Opp_3Q','Diff_3Q'])
BXSC_TM_OPP_DIFF_3Q

# Gm Stats
BXSC_OPP_GM = pd.DataFrame(data=BXSC_TEAM_GM.copy().loc[OPP_IDX].values,index=BXSC_TEAM_GM.index,columns=BXSC_TEAM_GM.columns)
BXSC_DIFF_GM = BXSC_TEAM_GM - BXSC_OPP_GM
BXSC_TM_OPP_DIFF_GM = pd.concat([BXSC_TEAM_GM,BXSC_OPP_GM,BXSC_DIFF_GM],axis=1,keys=['Team_GM','Opp_GM','Diff_GM'])
BXSC_TM_OPP_DIFF_GM

BXSC_TM_OPP_DIFF_GM_3Q = pd.concat([BXSC_TM_OPP_DIFF_GM,BXSC_TM_OPP_DIFF_3Q],axis=1)
BXSC_TM_OPP_DIFF_GM_3Q

BXSC_PLYR_TM_2ND_HALF = pd.concat([BXSC_PLYR_GM,BXSC_PLYR_3Q,BXSC_PLYR_H2,BXSC_PLYR_Q3,BXSC_PLYR_Q4,],axis=1,keys=['GM','3Q','H2','Q3','Q4'])
BXSC_PLYR_TM_2ND_HALF = BXSC_PLYR_TM_2ND_HALF.loc[BXSC_PLYR_TM_1ST_HALF.index]
BXSC_PLYR_TM_2ND_HALF = pd.merge(BXSC_PLYR_TM_2ND_HALF.reset_index(), BXSC_TM_OPP_DIFF_GM_3Q.reset_index(), on=['boxscore_id', 'team_id'],how='inner').set_index(['boxscore_id', 'team_id','player_id'])
BXSC_PLYR_TM_2ND_HALF

GM   
                                                                                    mp   
boxscore_id                  team_id              player_id                              
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/smithja05.html  31.016667  \
                                                  /players/c/chrisjo01.html  30.250000   
                                                  /players/g/greenja05.html  27.150000   
                                                  /players/s/sengual01.html  23.533333   
                                                  /players/m/martike04.html  21.033333   
                                                  /players/e/easonta01.html  26.966667   
                                                  /players/w/washity02.html  20.850000   
                                                  /players/n/nixda01.html    17.750000   
                                                  /players/k/kaminfr01.html  16.983333   
                                                  /players/m/marjabo01.html  12.633333   
                                                  /players/g/garubus01.html  11.833333   
                                                  /players/a/augusdj01.html   0.000000   
                                                  /players/c/caulewi01.html   0.000000   
                                                  /players/d/daysda01.html    0.000000   
                                                  /players/h/hudgitr01.html   0.000000   
...                                                                                ...   
/boxscores/199611010BOS.html /teams/CHI/1997.html /players/k/kerrst01.html   20.983333   
                                                  /players/w/wennibi01.html  10.133333   
                                                  /players/p/parisro01.html   8.716667   
                                                  /players/b/brownra02.html   5.383333   
                                                  /players/s/simpkdi01.html   4.700000   
                                                  /players/b/buechju01.html   0.616667   
                             /teams/BOS/1997.html /players/w/wesleda01.html  40.783333   
                                                  /players/f/foxri01.html    37.383333   
                                                  /players/e/ellispe01.html  31.133333   
                                                  /players/w/walkean02.html  26.583333   
                                                  /players/w/willier01.html  25.400000   
                                                  /players/b/barroda01.html  37.716667   
                                                  /players/d/dayto01.html    20.250000   
                                                  /players/b/brickfr01.html  18.883333   
                                                  /players/s/szabobr01.html   1.866667   

                                                                                   
                                                                              fg   
boxscore_id                  team_id              player_id                        
/boxscores/202304090WAS.html /teams/HOU/2023.html /players/s/smithja05.html  6.0  \
                                                  /players/c/chrisjo01.html  6.0   
                                                  /players/g/greenja05.html  5.0   
                                                  /players/s/sengual01.html  4.0   
                                                  /players/m/martike04.html  5.0   
                                                  /players/e/easonta01.html  6.0   
                                                  /players/w/washity02.html  3.0   
                                                  /players/n/nixda01.html    1.0   
                                                  /players/k/kaminfr01.html  3.0   
               

In [9]:
make_directory('./07-data-compiled/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023')
BXSC_PLYR_TM_1ST_HALF.to_csv('./07-data-compiled/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023/boxscores_players_1st_half.csv')
BXSC_PLYR_TM_2ND_HALF.to_csv('./07-data-compiled/03-boxscores-players-stats-1st-vs-2nd-half-1997-2023/boxscores_players_2nd_half.csv')

In [ ]:
X =  pd.read_csv('./03-data-aggregated/boxscores/boxscores_players_H1.csv', index_col=[2,0,1]).sort_index()
y = pd.read_csv('./03-data-aggregated/boxscores/boxscores_players_G.csv', index_col=[2,0,1])
y = y.loc[X.index] # Since quarter-by-quarter data is not available until 1996/1997, we only select the years for which we have both types of data
X

In [ ]:
y

In [ ]:
# We will only predict games where the player played at least 20 minutes
X['mp'].plot.hist()
y['mp'].plot.hist()
X1 = X[(X['mp'] > 5) & (y['mp'] >= 20)]
y1 = y[(X['mp'] > 5) & (y['mp'] >= 20)]
# y['mp'].plot.hist(bins=50,color='orange')

In [ ]:
X1_season = X1.index.to_frame()['team_id'].str[-9:-5].astype(int)
y1_season = y1.index.to_frame()['team_id'].str[-9:-5].astype(int)
X1_season

In [ ]:
player = '/players/j/jamesle01.html' #'/players/d/duranke01.html'#  # '/players/c/curryst01.html'
X2 = X1.loc[(player,slice(None),slice(None)),:][X1_season >= 2012].dropna(axis=0,how='any')
y2 = y1.loc[X2.index,'pts']
X2

In [ ]:
model_stack = ModelStackCV(ModelStackCV.MODELS_REGRESSION, ModelStackCV.MODEL_PARAMS_REGRESSION)
predictions = model_stack.fit_predict(X2, y2, cv=3)
scores = model_stack.score(predictions, ModelStackCV.SCORES_REGRESSION)
scores


In [ ]:
predictions_val = get_predictions_validation_set(predictions)
predictions_val

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))
plt.plot([0, 60], [0, 60],'-.', linewidth=2, color='gray')
plt.xlim(0,60)
plt.ylim(0,60)
# Draw a diagonal line
sns.scatterplot(y=predictions_val['y_true'], x=predictions_val['SVR'])

In [ ]:
predictions_val

In [ ]:
Z0 = predictions_val['y_true'].astype(int)
Z1 = predictions_val.iloc[:,1:].mean(axis=1).astype(int)

In [ ]:
plt.figure(figsize=(6,6))
plt.plot([0, 60], [0, 60],'-.', linewidth=2, color='gray')
plt.xlim(0,60)
plt.ylim(0,60)
# Draw a diagonal line
sns.scatterplot(x=Z1, y=Z0)

In [ ]:
Z3 = pd.DataFrame(index = Z1.value_counts().sort_index().index,columns=['over','under'])
# For each value in Z1, see if the corresponding value in Z0 is over or under
for val in Z1.value_counts().sort_index().index:
    Z4 = Z0[Z1 == val]
    Z3.loc[val,'over'] = Z4[Z4 >= val].count()
    Z3.loc[val,'under'] = Z4[Z4 < val].count()

Z3['over%'] = Z3['over'] / Z3.sum(axis=1)
Z3


In [ ]:
plt.figure(figsize=(6,6))
plt.plot([Z3.index.min(), Z3.index.max()], [0.5, 0.5],'-', linewidth=2, color='gray')
plt.xlim(Z3.index.min(), Z3.index.max())
plt.ylim(0,1)
Z3['over%'].plot.line()

In [ ]:
plt.figure(figsize=(10,8)) 
ax = sns.boxplot(x=Z1, y=Z0)


# Draw a diagonal line


In [ ]:
Z5 = (Z0 - Z1).to_frame()
Z5.rename(columns={0:'d'}, inplace=True)

Z5['o4-'] = Z5['d'] >= -4
Z5['o2-'] = Z5['d'] >= -2
# Z5['u'] = Z5['d'] < 0
Z5['o'] = Z5['d'] >= 0
Z5['o2+'] = Z5['d'] >= 2
Z5['o4+'] = Z5['d'] >= 4

Z5.iloc[:,1:].sum(axis=0) / Z5.shape[0]

In [ ]:
Z5['d'].plot.hist(bins=30)

In [ ]:
Z5['d'][Z5['d'] >= 0].mean()
Z5['d'][Z5['d'] < 0].mean()

In [ ]:
Z5['d'].abs().mean()

In [ ]:
0.77/6

In [ ]:
0.23/2

In [ ]:
Z5['d'][Z5['d'] >= 0].count()/Z5.shape[0] / Z5['d'][Z5['d'] >= 0].mean()
Z5['d'][Z5['d'] < 0].count()/Z5.shape[0] / Z5['d'][Z5['d'] < 0].mean()

In [ ]:
X.loc[('/players/d/duranke01.html','/boxscores/202311170UTA.html','/teams/PHO/2024.html'),:]